In [1]:
import sys
import logging
import importlib
importlib.reload(logging)  # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                    level=logging.INFO, stream=sys.stdout)

In [2]:
import mne
from mne.io import concatenate_raws

# 5,6,7,10,13,14 are codes for executed and imagined hands/feet
subject_id = 22  # carefully cherry-picked to give nice results on such limited data :)
event_codes = [5, 6, 9, 10, 13, 14]
# event_codes = [3,4,5,6,7,8,9,10,11,12,13,14]

# This will download the files if you don't have them yet,
# and then return the paths to the files.
physionet_paths = mne.datasets.eegbci.load_data(subject_id, event_codes)

# Load each of the files
parts = [mne.io.read_raw_edf(path, preload=True, stim_channel='auto',
                             verbose='WARNING')
         for path in physionet_paths]

# Concatenate them
raw = concatenate_raws(parts)

# Find the events in this dataset
events, _ = mne.events_from_annotations(raw)

# Use only EEG channels
eeg_channel_inds = \
    mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Extract trials, only using EEG channels
epoched = mne.Epochs(raw, events, dict(hands_or_left=2, feet_or_right=3),
                     tmin=1, tmax=4.1, proj=False, picks=eeg_channel_inds,
                     baseline=None, preload=True)

Used Annotations descriptions: ['T0', 'T1', 'T2']
90 matching events found
No baseline correction applied
Not setting metadata
Loading data for 90 events and 497 original time points ...
0 bad epochs dropped


In [3]:

import numpy as np
X = (epoched.get_data() * 1e6).astype(np.float32)
y = (epoched.events[:, 2] - 2).astype(np.int64)  # 2,3 -> 0,1


from braindecode.datautil.signal_target import SignalAndTarget

# in skorch flow it is also a validation set (train-valid split set in skorch model)
# To not shuffle data for train-valid split we need to create a splitter
train_set = SignalAndTarget(X[:70], y=y[:70])
# valid_set = SignalAndTarget(X[40:70], y=y[40:70])

In [4]:

from braindecode.models import ShallowFBCSPNet
from braindecode.torch_ext.util import set_random_seeds  # XXX : move to braindecode.util

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]

In [5]:
from skorch.classifier import NeuralNet, NeuralNetClassifier
from skorch.dataset import CVSplit
from torch.utils.data import Dataset

/home/maciej/.virtualenvs/braindecode/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
class EEGDataSet(Dataset):
    def __init__(self, X, y):
        self.X = X.reshape(-1, 64, 497, 1)
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [7]:
train_set = EEGDataSet(train_set.X, train_set.y)

In [8]:
from torch import optim

In [9]:
import torch
import torch.nn.functional as F

In [10]:
class Splitter:
    def __init__(self, train_size):
        assert isinstance(train_size, (int, float))
        self.train_size = train_size
        
    def __call__(self, dataset, y, **kwargs):
        if isinstance(self.train_size, int):
            dataset_train = EEGDataSet(dataset.X[:self.train_size],dataset.y[:self.train_size])
            dataset_valid = EEGDataSet(dataset.X[self.train_size:], dataset.y[self.train_size:])
#             dataset_train = torch.utils.data.Subset(dataset, range(0, self.train_size))
#             dataset_valid = torch.utils.data.Subset(dataset, range(self.train_size, len(X)))
        elif isinstance(self.train_size, float):
            raise Exception
            dataset_train = torch.utils.data.Subset(dataset, range(
                0, int(self.train_size * len(dataset))))
            dataset_valid = torch.utils.data.Subset(dataset, range(
                int(self.train_size * len(dataset)), len(X)))
        return dataset_train, dataset_valid

In [15]:
# final_conv_length = auto ensures we only get a single output in the time dimension
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=train_set.X.shape[2],
                        final_conv_length='auto').create_network()
if cuda:
    model.cuda()

# It can use also NeuralNetClassifier
skorch_model = NeuralNet(model,
                         criterion=torch.nn.NLLLoss,
                         optimizer=optim.Adam, train_split=Splitter(40),
                         optimizer__lr=0.0625 * 0.01, optimizer__weight_decay=0, batch_size=64)
        
#                                   )#Splitter(40))#, callbacks=[('accuracy', EpochScoring('accuracy', lower_is_better=False))])

In [16]:
skorch_model.fit(train_set, y=None, epochs=4)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        1.6128        0.8567  0.7583
      2        0.6304        1.0264  0.5937
      3        0.3136        0.8096  0.6120
      4        0.4003        0.6471  0.6609


<class 'skorch.net.NeuralNet'>[initialized](
  module_=Sequential(
    (dimshuffle): Expression(expression=_transpose_time_to_spat)
    (conv_time): Conv2d(1, 40, kernel_size=(25, 1), stride=(1, 1))
    (conv_spat): Conv2d(40, 40, kernel_size=(1, 64), stride=(1, 1), bias=False)
    (bnorm): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_nonlin): Expression(expression=square)
    (pool): AvgPool2d(kernel_size=(75, 1), stride=(15, 1), padding=0)
    (pool_nonlin): Expression(expression=safe_log)
    (drop): Dropout(p=0.5, inplace=False)
    (conv_classifier): Conv2d(40, 2, kernel_size=(27, 1), stride=(1, 1))
    (softmax): LogSoftmax()
    (squeeze): Expression(expression=_squeeze_final_output)
  ),
)

In [14]:
# from braindecode.torch_ext.optimizers import AdamW
from torch.optim import Adam
import torch.nn.functional as F
# optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = Adam(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
model.compile(loss=F.nll_loss, optimizer=optimizer, iterator_seed=1,)

# ## Run the training

# In[ ]:


n_epochs = 4
model.fit(train_set.X, train_set.y, n_epochs=n_epochs, batch_size=64,
          scheduler='cosine', validation_data=(valid_set.X, valid_set.y))

AttributeError: 'Sequential' object has no attribute 'compile'

In [ ]:
# The monitored values are also stored into a pandas dataframe:

model.epochs_df

# Eventually, we arrive at 83.4% accuracy, so 25 from 30 trials are correctly
# predicted. In the [Cropped Decoding Tutorial](./Cropped_Decoding.html),
# we can learn how to achieve higher accuracies using cropped training.

##############################################################################
# Evaluation
# ----------

# Once we have all our hyperparameters and architectural choices done, we
# can evaluate the accuracies to report in our publication by evaluating on
# the test set:

test_set = SignalAndTarget(X[70:], y=y[70:])

model.evaluate(test_set.X, test_set.y)

# We can also retrieve predicted labels per trial as such:

y_pred = model.predict(test_set.X)

# We can also retrieve the raw network outputs per trial as such:
#
# <div class="alert alert-warning">
# Note these are log-softmax outputs, so to get probabilities one would
# have to exponentiate them using `th.exp`.
# </div>

model.predict_outs(test_set.X)

# <div class="alert alert-info">
#
# If you want to try cross-subject decoding, changing the loading code to
# the following will perform cross-subject decoding on imagined left vs
# right hand closing, with 50 training and 5 validation subjects
# (Warning, might be very slow if you are on CPU):
#
# </div>

from braindecode.datautil import SignalAndTarget

# First 50 subjects as train
physionet_paths = [mne.datasets.eegbci.load_data(sub_id, [4, 8, 12])
                   for sub_id in range(1, 51)]
physionet_paths = np.concatenate(physionet_paths)
parts = [mne.io.read_raw_edf(path, preload=True, stim_channel='auto')
         for path in physionet_paths]

raw = concatenate_raws(parts)

picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False,
                       eog=False, exclude='bads')

# Find the events in this dataset
events, _ = mne.events_from_annotations(raw)

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epoched = mne.Epochs(raw, events, dict(hands=2, feet=3), tmin=1,
                     tmax=4.1, proj=False, picks=picks,
                     baseline=None, preload=True)

# 51-55 as validation subjects
physionet_paths_valid = [mne.datasets.eegbci.load_data(sub_id, [4, 8, 12])
                         for sub_id in range(51, 56)]
physionet_paths_valid = np.concatenate(physionet_paths_valid)
parts_valid = [mne.io.read_raw_edf(path, preload=True, stim_channel='auto')
               for path in physionet_paths_valid]
raw_valid = concatenate_raws(parts_valid)

picks_valid = mne.pick_types(raw_valid.info, meg=False, eeg=True,
                             stim=False, eog=False,
                             exclude='bads')

events_valid, _ = mne.events_from_annotations(raw_valid)

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epoched_valid = mne.Epochs(raw_valid, events_valid, dict(hands=2, feet=3),
                           tmin=1, tmax=4.1, proj=False, picks=picks_valid,
                           baseline=None, preload=True)

train_X = (epoched.get_data() * 1e6).astype(np.float32)
train_y = (epoched.events[:, 2] - 2).astype(np.int64)  # 2,3 -> 0,1
valid_X = (epoched_valid.get_data() * 1e6).astype(np.float32)
valid_y = (epoched_valid.events[:, 2] - 2).astype(np.int64)  # 2,3 -> 0,1
train_set = SignalAndTarget(train_X, y=train_y)
valid_set = SignalAndTarget(valid_X, y=valid_y)

In [ ]:
compare_df

In [ ]:
exp.epochs_df